In [1]:
import glob
import pandas as pd
from feature_engine.datetime import DatetimeFeatures
from sklearn.linear_model import LinearRegression, TweedieRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import set_config
set_config(display='diagram')

In [2]:
files = glob.glob('data/*.xlsx')	
nrows = [30, 31, 30, 31, 31, 30, 31]
files

['data\\04 April 2017 Cyclist Numbers for Web.xlsx',
 'data\\05 May 2017 Cyclist Numbers for Web.xlsx',
 'data\\06 June 2017 Cyclist Numbers for Web.xlsx',
 'data\\07 July 2017 Cyclist Numbers for Web.xlsx',
 'data\\08 August 2017 Cyclist Numbers for Web.xlsx',
 'data\\09 September 2017 Cyclist Numbers for Web.xlsx',
 'data\\10 October 2017 Cyclist Numbers.xlsx']

In [3]:
df = pd.concat([pd.read_excel(file, skiprows=[0,1,2,3], header= 1, nrows=nrow) for file, nrow in zip(files,nrows)]).drop(columns=['Unnamed: 0', 'Unnamed: 11','Day'])
df.dtypes
df.sort_values(by=['Date'])

,Date,High Temp (°F),Low Temp (°F),Precipitation,Brooklyn Bridge,Manhattan Bridge,Williamsburg Bridge,Queensboro Bridge,Total
0,2017-04-01,46.0,37.0,0,606,1446,1915,1430,5397
1,2017-04-02,62.1,41.0,0,2021,3943,4207,2862,13033
2,2017-04-03,63.0,50.0,0.03,2470,4988,5178,3689,16325
3,2017-04-04,51.1,46.0,1.18,723,1913,2279,1666,6581
4,2017-04-05,63.0,46.0,0,2807,5276,5711,4197,17991
...,...,...,...,...,...,...,...,...,...
26,2017-10-27,62.1,48.0,0,3150,5610,6450,5181,20391
27,2017-10-28,68.0,55.9,0,2245,4520,5104,4069,15938
28,2017-10-29,64.9,61.0,3.03,183,661,1026,965,2835
29,2017-10-30,55.0,46.0,0.25,1428,2966,3547,2924,10865


In [4]:
df.dtypes

Date                   datetime64[ns]
High Temp (°F)                float64
Low Temp (°F)                 float64
Precipitation                  object
Brooklyn Bridge                 int64
Manhattan Bridge                int64
Williamsburg Bridge             int64
Queensboro Bridge               int64
Total                           int64
dtype: object

In [5]:
df['Date'] = df['Date'].astype('datetime64')
df['High Temp (°F)'] = df['High Temp (°F)'].astype('float')
df['Low Temp (°F)'] = df['Low Temp (°F)'].astype('float')
cols = df.filter(like='Bridge').columns
df[cols] = df[cols].astype('int64')
df['Total'] = df['Total'].astype('int64')
df.dtypes

Date                   datetime64[ns]
High Temp (°F)                float64
Low Temp (°F)                 float64
Precipitation                  object
Brooklyn Bridge                 int64
Manhattan Bridge                int64
Williamsburg Bridge             int64
Queensboro Bridge               int64
Total                           int64
dtype: object

In [6]:
(df.filter(like='Bridge').sum(axis=1) == df.Total).all()

True

In [7]:
df['Precipitation'] = df.Precipitation.astype('str').str.replace('T|S', '0', regex=True).astype('float')

In [8]:
df.dtypes

Date                   datetime64[ns]
High Temp (°F)                float64
Low Temp (°F)                 float64
Precipitation                 float64
Brooklyn Bridge                 int64
Manhattan Bridge                int64
Williamsburg Bridge             int64
Queensboro Bridge               int64
Total                           int64
dtype: object

In [9]:
X = df.drop(columns = cols).drop(columns = ['Total'])
y = df.Total

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [11]:
dtf = DatetimeFeatures(features_to_extract=['week', 'month',  'day_of_month', 'day_of_year'])
# dtf.fit_transform(X_train).dtypes

In [12]:
pipe = Pipeline(steps=[
    ('dtf', DatetimeFeatures(features_to_extract=['day_of_year', 'day_of_month', 'month', 'week'])),
    ('model', TweedieRegressor(power=1, link = 'log', max_iter=10000))  # distribución del total es una distribución gamma (valor/evento). Mixed poisson y gamma. ]1,2[ mixed poisson y gamma. 1: Poisson, 2: Gamma
])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

mean_squared_error(y_test, y_pred, squared=False)

3543.848884011572

In [13]:
y.mean(), y.min(), y.max()

(18627.67757009346, 2374, 26969)

In [14]:
pipe = Pipeline(steps=[
    ('dtf', DatetimeFeatures(features_to_extract=[ 'day_of_year', 'day_of_month', 'month', 'week'])),
    # distribución del total es una distribución gamma (valor/evento). Mixed poisson y gamma. ]1,2[ mixed poisson y gamma. 1: Poisson, 2: Gamma
    ('model', LinearRegression())
])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

mean_squared_error(y_test, y_pred, squared=False)


4001.1751994690817

In [15]:
from sklearn.model_selection import cross_val_score
pipe = Pipeline(steps=[
    ('dtf', DatetimeFeatures(features_to_extract=[ 'day_of_year', 'day_of_month', 'month', 'week'])),
    ('model', TweedieRegressor(power=1, link = 'log', max_iter=10000))
])

-cross_val_score(pipe, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error').mean()

3300.9925314906695

In [16]:
pipe = Pipeline(steps=[
    ('dtf', DatetimeFeatures(features_to_extract=['day_of_year', 'day_of_month', 'month', 'week'])),
    ('model', LinearRegression())
])

-cross_val_score(pipe, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error').mean()


3798.22032976436